In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA2_Pg/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [8]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
5679,71.771558,0.338368,14.666163,1.936155,1.558040,0.080668,0.653298,1.494996,3.656076,3.844678
5680,71.780197,0.337888,14.662852,1.936383,1.555776,0.080614,0.651776,1.491916,3.656173,3.846426
5681,71.788930,0.337403,14.659499,1.936613,1.553487,0.080559,0.650238,1.488805,3.656270,3.848196
6029,71.761119,0.338948,14.670156,1.935881,1.560777,0.080734,0.655139,1.498721,3.655959,3.842569
6030,71.770293,0.338438,14.666647,1.936122,1.558372,0.080676,0.653520,1.495447,3.656062,3.844422


In [9]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
5679,71.771558,0.338368,14.666163,1.936155,1.558040,0.080668,0.653298,1.494996,3.656076,3.844678
5680,71.780197,0.337888,14.662852,1.936383,1.555776,0.080614,0.651776,1.491916,3.656173,3.846426
5681,71.788930,0.337403,14.659499,1.936613,1.553487,0.080559,0.650238,1.488805,3.656270,3.848196
6029,71.761119,0.338948,14.670156,1.935881,1.560777,0.080734,0.655139,1.498721,3.655959,3.842569
6030,71.770293,0.338438,14.666647,1.936122,1.558372,0.080676,0.653520,1.495447,3.656062,3.844422
...,...,...,...,...,...,...,...,...,...,...
235415,70.834720,0.377321,15.134783,1.254007,2.089758,0.081016,0.937756,2.080194,3.640578,3.569868
235416,69.591060,0.445259,15.418800,1.390627,2.350768,0.088601,1.171717,2.523916,3.615410,3.403842
235417,69.589145,0.445370,15.419180,1.390931,2.351118,0.088613,1.172078,2.524584,3.615368,3.403613
235418,69.587270,0.445478,15.419551,1.391231,2.351460,0.088626,1.172430,2.525238,3.615328,3.403389


In [10]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45001:950000]

In [11]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA2_Pg/area2_Pg_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA2_Pg/area2_Pg_2.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [12]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2_Pg/area2_Pg_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2_Pg/area2_Pg_2_results_excel.txt")

C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [13]:
df_final = pd.concat([df_final1, df_final2])

In [14]:
df_final

,Q,P,K
SAMPLE,,,
5679,31.223118,35.257417,25.812165
5680,31.235062,35.242823,25.822583
5681,31.247103,35.228108,25.833126
6029,31.208729,35.274960,25.799596
6030,31.221422,35.259497,25.810611
...,...,...,...
235415,29.077871,38.042377,24.175163
235416,27.194279,40.051954,23.172774
235417,27.191427,40.054920,23.171380


## QAPF classification

In [15]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [16]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite'], dtype=object)

In [17]:
df_final.to_excel("../_CIPW/CIPW/AREA2_Pg/qapf.xlsx", index=True)

-------

In [18]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA2_Pg/QAPF_counts.xlsx")

-----

In [19]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area2_Pg.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [20]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
37,688625.880248,5.439932e+06,53.0,T,2.0,39.748474,25.220485,31.978906,monzo granite
40,583524.998244,4.923464e+06,53.0,T,2.0,42.890640,23.404756,28.820672,monzo granite
44,861048.721020,5.826241e+06,53.0,T,2.0,38.295383,31.586014,27.933697,monzo granite
47,755477.668714,5.262227e+06,53.0,T,2.0,41.044247,25.818789,29.731042,monzo granite
52,798508.250913,5.526520e+06,53.0,T,2.0,42.548016,16.753596,36.667948,syeno granite
...,...,...,...,...,...,...,...,...,...
4630,824984.331911,5.905587e+06,53.0,T,2.0,6.071187,56.000177,15.931197,quartz monzodiorite\nquartz monzogabbro
4632,591400.162716,4.929279e+06,53.0,T,2.0,0.000000,67.357975,16.205206,monzodiorite monzogabbro
4644,820876.841657,5.613306e+06,53.0,T,2.0,2.915238,52.174453,19.896758,monzodiorite monzogabbro
4646,626698.329397,4.987292e+06,53.0,T,2.0,2.711446,54.229187,8.713380,monzodiorite monzogabbro


In [21]:
QAPF_control_area2_Pg = QAPF_control[QAPF_control['area'] ==2]
QAPF_control_area2_Pg.to_excel("../_CIPW/CIPW/AREA2_Pg/QAPF_control_points.xlsx", index=True)

In [23]:
QAPF_control_area2_Pg

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
37,688625.880248,5.439932e+06,53.0,T,2.0,39.748474,25.220485,31.978906,monzo granite
40,583524.998244,4.923464e+06,53.0,T,2.0,42.890640,23.404756,28.820672,monzo granite
44,861048.721020,5.826241e+06,53.0,T,2.0,38.295383,31.586014,27.933697,monzo granite
47,755477.668714,5.262227e+06,53.0,T,2.0,41.044247,25.818789,29.731042,monzo granite
52,798508.250913,5.526520e+06,53.0,T,2.0,42.548016,16.753596,36.667948,syeno granite
...,...,...,...,...,...,...,...,...,...
4630,824984.331911,5.905587e+06,53.0,T,2.0,6.071187,56.000177,15.931197,quartz monzodiorite\nquartz monzogabbro
4632,591400.162716,4.929279e+06,53.0,T,2.0,0.000000,67.357975,16.205206,monzodiorite monzogabbro
4644,820876.841657,5.613306e+06,53.0,T,2.0,2.915238,52.174453,19.896758,monzodiorite monzogabbro
4646,626698.329397,4.987292e+06,53.0,T,2.0,2.711446,54.229187,8.713380,monzodiorite monzogabbro


In [24]:
QAPF_control_area2_Pg.iloc[:, 8].value_counts().to_excel("../_CIPW/CIPW/AREA2_Pg/QAPF_counts_control_points.xlsx")